In [1]:
import pandas as pd
import collections
import skbio 
from skbio.diversity.alpha import shannon 
import IPython
import re
import numpy

In [2]:
vdjdb_data = pd.read_csv("vdjdb_data.csv")

In [3]:
del vdjdb_data ['Unnamed: 0']

In [4]:
vdjdb_data.head()

,Gene,CDR3,V,J,Species,MHC A,Epitope,Epitope gene
0,TRA,CAVGNNAGNMLTF,TRAV20*01,TRAJ39*01,HomoSapiens,HLA-B*44:03:08,NEGVKAAW,IE2
1,TRA,CAAKTGGGNKLTF,TRAV29/DV5*01,TRAJ10*01,HomoSapiens,HLA-B*44:03:08,NEGVKAAW,IE2
2,TRA,CAAKAGGGNKLTF,TRAV29/DV5*01,TRAJ10*01,HomoSapiens,HLA-B*44:03:08,NEGVKAAW,IE2
3,TRA,CAAITGIGGSQGNLIF,TRAV13-1*01,TRAJ42*01,HomoSapiens,HLA-B*44:03:08,NEGVKAAW,IE2
4,TRA,CAGTDGGATNKLIF,TRAV36/DV7*01,TRAJ32*01,HomoSapiens,HLA-B*44:03:08,NEGVKAAW,IE2


In [5]:
TRAJ = vdjdb_data ['J'].unique()
TRAV = vdjdb_data ['V'].unique()

In [6]:
print (TRAJ) # via deze methode gekeken hoe de data eruitzag om te splitsen

['TRAJ39*01' 'TRAJ10*01' 'TRAJ42*01' 'TRAJ32*01' 'TRAJ13*01' 'TRAJ8*01'
 'TRAJ47*01' 'TRAJ29*01' 'TRAJ17*01' 'TRAJ38*01' 'TRAJ44*01' 'TRAJ49*01'
 'TRAJ26*01' 'TRAJ34*01' 'TRAJ40*01' 'TRAJ33*01' 'TRAJ23*01' 'TRAJ57*01'
 'TRAJ43*01' 'TRAJ6*01' 'TRAJ20*01' 'TRAJ30*01' 'TRAJ48*01' 'TRAJ31*01'
 'TRAJ45*01' 'TRAJ3*01' 'TRAJ27*01' 'TRAJ36*01' 'TRAJ52*01' 'TRAJ15*01'
 'TRAJ5*01' 'TRAJ4*01' 'TRAJ50*01' 'TRAJ28*01' 'TRAJ53*01' 'TRAJ37*01'
 'TRAJ22*01' 'TRAJ41*01' 'TRAJ9*01' 'TRAJ12*01' 'TRAJ11*01' 'TRAJ46*01'
 'TRAJ54*01' 'TRAJ18*01' 'TRAJ21*01' 'TRAJ56*01' 'TRAJ7*01' 'TRAJ14*01'
 'TRAJ13*02' 'TRAJ24*02']


In [7]:
print (TRAV) # hier is wat meer werk aan 

['TRAV20*01' 'TRAV29/DV5*01' 'TRAV13-1*01' 'TRAV36/DV7*01' 'TRAV19*01'
 'TRAV8-2*01' 'TRAV27*01' 'TRAV8-6*01' 'TRAV35*01' 'TRAV9-2*01'
 'TRAV23/DV6*01' 'TRAV38-2/DV8*01' 'TRAV18*01' 'TRAV8-3*01' 'TRAV22*01'
 'TRAV13-2*01' 'TRAV21*01' 'TRAV8-4*01' 'TRAV5*01' 'TRAV12-2*01'
 'TRAV38-1*01' 'TRAV12-1*01' 'TRAV1-2*01' 'TRAV39*01' 'TRAV16*01'
 'TRAV17*01' 'TRAV10*01' 'TRAV24*01' 'TRAV41*01' 'TRAV12-3*01' 'TRAV3*01'
 'TRAV8-1*01' 'TRAV26-2*01' 'TRAV4*01' 'TRAV14/DV4*01' 'TRAV2*01'
 'TRAV30*01' 'TRAV25*01' 'TRAV26-1*01' 'TRAV1-1*01' 'TRAV34*01'
 'TRAV40*01' 'TRAV6*01' 'TRAV9-1*01' 'TRAV7*01' 'TRAV14/DV4*02'
 'TRAV8-6*02']


In [8]:
vdjdb_data = vdjdb_data[sorted(vdjdb_data.columns)]

In [9]:
vdjdb_data.head()

,CDR3,Epitope,Epitope gene,Gene,J,MHC A,Species,V
0,CAVGNNAGNMLTF,NEGVKAAW,IE2,TRA,TRAJ39*01,HLA-B*44:03:08,HomoSapiens,TRAV20*01
1,CAAKTGGGNKLTF,NEGVKAAW,IE2,TRA,TRAJ10*01,HLA-B*44:03:08,HomoSapiens,TRAV29/DV5*01
2,CAAKAGGGNKLTF,NEGVKAAW,IE2,TRA,TRAJ10*01,HLA-B*44:03:08,HomoSapiens,TRAV29/DV5*01
3,CAAITGIGGSQGNLIF,NEGVKAAW,IE2,TRA,TRAJ42*01,HLA-B*44:03:08,HomoSapiens,TRAV13-1*01
4,CAGTDGGATNKLIF,NEGVKAAW,IE2,TRA,TRAJ32*01,HLA-B*44:03:08,HomoSapiens,TRAV36/DV7*01


In [10]:
#for gene in [gene for gene in data.columns.values if 'gene' in gene]:
    # split compound gene names (only really applicable to the V gene for the peptide data)
 #   data = split_rows(data, gene, '/')
    # uniformly encode all gene names
  #  data[gene] = data[gene].apply(lambda x: encode_gene(x))
    # explicitly add the gene family
   # data[gene.replace('gene', 'family')] = data[gene].apply(lambda x: gene_to_family(x))

#data = data[sorted(data.columns)]
# sorteren op alfabet""

In [11]:
vdjdb_data.replace('TRAJ','',regex=True, inplace = True)
vdjdb_data[['J_family','J_allele']] = vdjdb_data.J.str.split ("*",expand=True)
del vdjdb_data ['J']
vdjdb_data.head()

,CDR3,Epitope,Epitope gene,Gene,MHC A,Species,V,J_family,J_allele
0,CAVGNNAGNMLTF,NEGVKAAW,IE2,TRA,HLA-B*44:03:08,HomoSapiens,TRAV20*01,39,01
1,CAAKTGGGNKLTF,NEGVKAAW,IE2,TRA,HLA-B*44:03:08,HomoSapiens,TRAV29/DV5*01,10,01
2,CAAKAGGGNKLTF,NEGVKAAW,IE2,TRA,HLA-B*44:03:08,HomoSapiens,TRAV29/DV5*01,10,01
3,CAAITGIGGSQGNLIF,NEGVKAAW,IE2,TRA,HLA-B*44:03:08,HomoSapiens,TRAV13-1*01,42,01
4,CAGTDGGATNKLIF,NEGVKAAW,IE2,TRA,HLA-B*44:03:08,HomoSapiens,TRAV36/DV7*01,32,01


In [12]:
vdjdb_data.replace('TRAV','',regex=True, inplace = True)
vdjdb_data[['V','V_allele']] = vdjdb_data.V.str.split ("*",expand=True)
#vdjdb_data[['V_family','V_gene']] = vdjdb_data.V.str.split ("/",expand=True)
#vdjdb_data_split = vdjdb_data[['V_family','V_gene']] = vdjdb_data.V.str.split ("-",expand=True)

In [13]:
vdjdb_data.head()

,CDR3,Epitope,Epitope gene,Gene,MHC A,Species,V,J_family,J_allele,V_allele
0,CAVGNNAGNMLTF,NEGVKAAW,IE2,TRA,HLA-B*44:03:08,HomoSapiens,20,39,01,01
1,CAAKTGGGNKLTF,NEGVKAAW,IE2,TRA,HLA-B*44:03:08,HomoSapiens,29/DV5,10,01,01
2,CAAKAGGGNKLTF,NEGVKAAW,IE2,TRA,HLA-B*44:03:08,HomoSapiens,29/DV5,10,01,01
3,CAAITGIGGSQGNLIF,NEGVKAAW,IE2,TRA,HLA-B*44:03:08,HomoSapiens,13-1,42,01,01
4,CAGTDGGATNKLIF,NEGVKAAW,IE2,TRA,HLA-B*44:03:08,HomoSapiens,36/DV7,32,01,01


In [16]:
vdjdb_pos = vdjdb_data.groupby('Epitope').filter(lambda x: len(x)> 30)


In [18]:
## Data exploration
classes = vdjdb_pos['Epitope'].unique()
stats = collections.defaultdict(list)
for peptide in classes:
    pep_data = vdjdb_pos[vdjdb_pos['Epitope'] == peptide]
    
    stats['total TCRAs'].append(pep_data.shape[0])
    stats['unique CDR3'].append(len(set(pep_data['CDR3'])))
    
    cdr3_count = list(collections.Counter(pep_data['CDR3']).values())
    stats['CDR3 diversity'].append(shannon(cdr3_count))
    
    #stats['unique V family'].append(len(pep_data['V_family'].unique()))
    stats['unique J family'].append(len(pep_data['J_family'].unique()))
    #stats['unique V gene'].append(len(pep_data['V_gene'].unique()))
    stats['unique J allele'].append(len(pep_data['J_allele'].unique()))
    stats['unique V allele'].append(len(pep_data['V_allele'].unique()))
    #v_fam_count = list(collections.Counter(pep_data['V_family']).values())
    j_fam_count = list(collections.Counter(pep_data['J_family']).values())
    #v_gene_count = list(collections.Counter(pep_data['V_gene']).values())
    j_allele_count = list(collections.Counter(pep_data['J_allele']).values())
    v_allele_count = list(collections.Counter(pep_data['V_allele']).values())
    #stats['V family diversity'].append(shannon(v_fam_count))
    stats['J family diversity'].append(shannon(j_fam_count))
    #stats['V gene diversity'].append(shannon(v_gene_count))
    stats['J allele diversity'].append(shannon(j_allele_count))
    stats['V allele diversity'].append(shannon(v_allele_count))

IPython.display.display(pd.DataFrame(stats, index=classes, columns=stats.keys()).T)
# Unique J en V allele zeggen hier niet zo veel omdat deze bijna altijd alleen 01 of 02 zijn

,NEGVKAAW,KLGGALQAK,ELAGIGILTV,IVTDFSVIK,AVFDRKSDAK,RAKFKQLL,GILGFVFTL,SLFNTVATLY,RLRAEAQVK,AYAQKIFKI,...,GLIYNRMGAVTTEV,PKYVKQNTLKLAT,QARQMVQAMRTIGTHP,SGPLKAEIAQRLED,GMFNMLSTVLGVS,FRDYVDRFYKTLRAEQASQE,LLLGIGILV,KAFSPEVIPMF,DATYQRTRALVR,YLQPRTFLL
total TCRAs,301.000000,13401.000000,361.000000,710.000000,1687.000000,1213.000000,4472.000000,37.000000,413.000000,39.000000,...,128.000000,141.000000,111.000000,63.000000,72.000000,248.000000,376.000000,31.000000,112.000000,325.000000
unique CDR3,118.000000,11426.000000,332.000000,519.000000,1534.000000,642.000000,3004.000000,37.000000,390.000000,39.000000,...,119.000000,139.000000,102.000000,62.000000,65.000000,141.000000,345.000000,22.000000,88.000000,262.000000
CDR3 diversity,6.159650,13.212909,8.318918,8.044908,10.433803,8.097860,10.818358,5.209453,8.563207,5.285402,...,6.859375,7.111183,6.542164,5.945534,5.975481,6.660757,8.380361,4.324849,6.322849,7.817597
unique J family,38.000000,48.000000,46.000000,46.000000,47.000000,46.000000,48.000000,25.000000,46.000000,24.000000,...,37.000000,42.000000,31.000000,31.000000,34.000000,27.000000,43.000000,9.000000,35.000000,37.000000
unique J allele,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,2.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,2.000000,1.000000,1.000000,1.000000,1.000000
unique V allele,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,2.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
J family diversity,4.477435,5.390525,5.013560,5.007671,5.392408,4.711714,4.246334,4.465946,5.320349,4.355539,...,4.827183,5.110287,4.383797,4.677688,4.657510,3.357188,4.834619,2.382494,4.711158,3.825526
J allele diversity,-0.000000,-0.000000,-0.000000,-0.000000,-0.000000,-0.000000,0.008039,-0.000000,-0.000000,-0.000000,...,-0.000000,-0.000000,-0.000000,-0.000000,-0.000000,0.067670,-0.000000,-0.000000,-0.000000,-0.000000
V allele diversity,-0.000000,-0.000000,-0.000000,-0.000000,-0.000000,-0.000000,0.024864,-0.000000,-0.000000,-0.000000,...,-0.000000,-0.000000,-0.000000,-0.000000,-0.000000,-0.000000,-0.000000,-0.000000,-0.000000,-0.000000


In [ ]:
## vanaf hier verder werken met de gemergde positieve data

In [ ]:
## Data exploration

# calculate CDR3 sequence diversity of input data

stats = collections.defaultdict(list)
for peptide in classes:
    pep_data = data[data['HLA_peptide'] == peptide]
    
    stats['total TCRBs'].append(pep_data.shape[0])
    stats['unique CDR3'].append(len(set(pep_data['CDR3_sequence'])))
    
    cdr3_count = list(collections.Counter(pep_data['CDR3_sequence']).values())
    stats['CDR3 diversity'].append(shannon(cdr3_count))
    
    stats['unique V family'].append(len(pep_data['V_family'].unique()))
    stats['unique J family'].append(len(pep_data['J_family'].unique()))
    stats['unique V gene'].append(len(pep_data['V_gene'].unique()))
    stats['unique J gene'].append(len(pep_data['J_gene'].unique()))
    
    v_fam_count = list(collections.Counter(pep_data['V_family']).values())
    j_fam_count = list(collections.Counter(pep_data['J_family']).values())
    v_gene_count = list(collections.Counter(pep_data['V_gene']).values())
    j_gene_count = list(collections.Counter(pep_data['J_gene']).values())
    stats['V family diversity'].append(shannon(v_fam_count))
    stats['J family diversity'].append(shannon(j_fam_count))
    stats['V gene diversity'].append(shannon(v_gene_count))
    stats['J gene diversity'].append(shannon(j_gene_count))

IPython.display.display(pd.DataFrame(stats, index=classes, columns=stats.keys()).T)